In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro, to_avro
import pandas as pd
import json

In [ ]:
#Spark Session creation configured to interact with Kafka
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-avro_2.12:3.0.0,org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
getOrCreate()

In [ ]:
#Read data from Kafka
data = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "ec2-34-217-75-40.us-west-2.compute.amazonaws.com:9092")\
  .option("subscribe", "twitter_test")\
  .option("startingOffsets", "earliest")\
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").alias("data").select("data.value")

In [ ]:
#write streaming data as a text file
data.\
writeStream.\
format("text").\
option("checkpointLocation", "checkpoint/schema").\
option("format", "text").\
option("path", "schema/in").\
outputMode("append").\
start()

In [ ]:
#extract schema by reading the file written above
smallBatchSchema = spark.read.json("schema/in/*.txt").schema

In [ ]:
#write schema as JSON file to schema folder
with open("schema/out/tweet_schema.json", "w") as f:
    json.dump(smallBatchSchema.jsonValue(), f)